### SEVER DEFINITIONS


In [1]:
# Necessary Imports
from flask import Flask, jsonify
from werkzeug.routing import IntegerConverter
from _csv import Dialect as _Dialect
import csv
import copy
import os
from rtls_slave import aoa_main, pixel_calculate


# Handle Negative Values
class SignedIntConverter(IntegerConverter):
    regex = r'-?\d+'
app = Flask(__name__)
app.url_map.converters['signed_int'] = SignedIntConverter
#Store input values: AoA and Horizontal Angle

aoabefore = {}
aoa= {}
lev = []

@app.route('/')
def home():
    return "This is the HomePage"

#Routing Settings
@app.route('/send/<signed_int:horiz>', methods=['GET'])
def get_angle(horiz):
    global aoa
    global aoabefore
    print("Plane Angle: " + str(horiz))
    aoa_result = aoa_main(devices=[{"com_port": "COM5", "baud_rate": 460800, "name": "CC26x2 Master"}])
    input_letter = input("If you want to add this data to list type 'y', otherwise press Enter")
    if input_letter!='y':
        return jsonify({'state':'Try Again'})
    lev.append(horiz)
    aoabefore = copy.deepcopy(aoa)
    aoa = aoa_result
        
    return jsonify({'state': 'success'})

@app.route('/calculate/', methods=['GET'])
def calculate():
    print("Before Turning："+str(aoabefore)+'\n'+"After Turning： " + str(aoa) +'\n'+"Rotation Angle： "+ str(lev))
    #aoa_image()
    u = []
    v = []

    set_aoa = set(aoa)
    set_aoabefore = set(aoabefore)

    for slave in set_aoa.intersection(set_aoabefore):
        minnumber = min(len(aoabefore[slave]), len(aoa[slave]))
        f = open(f"./aoa_results/{slave.replace(':','_')}_before.csv",'a',newline="")
        f.truncate()
        writer = csv.writer(f)
        

        for i in range(minnumber):
            for j in range(minnumber):
                (temp1, temp2) = pixel_calculate(lev[-2] - lev[-1], aoabefore[slave][i], aoa[slave][j])

                if (temp1 != 1):
                    u.append(temp1)
                    v.append(temp2)
                    tupx = (temp1,temp2)
                    writer.writerow(tupx)
        
        f.close()

    
    return jsonify({'state': 'success'})

### RUNNING THE SEVER

In [2]:
app.run(host='0.0.0.0',port = 5000, debug=False) 

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://10.19.144.53:5000
Press CTRL+C to quit


Plane Angle: -79
Master : <RTLSNode(CC26x2 Master, started daemon 1896)> 
Passives : [] 
All : [<RTLSNode(CC26x2 Master, started daemon 1896)>]
Devices Reset
Start scan for 5 sec
Scan Results: [{'addr': '80:6F:B0:EE:AA:13', 'addrType': 0, 'rssi': -71, 'advSID': 1, 'periodicAdvInt': 240}, {'addr': '80:6F:B0:EE:9C:67', 'addrType': 0, 'rssi': -60, 'advSID': 1, 'periodicAdvInt': 240}, {'addr': '80:6F:B0:EE:9C:67', 'addrType': 0, 'rssi': -60, 'advSID': 255, 'periodicAdvInt': 0}, {'addr': '80:6F:B0:EE:AA:13', 'addrType': 0, 'rssi': -63, 'advSID': 255, 'periodicAdvInt': 0}]
Try to connect to : 80:6F:B0:EE:AA:13
Connected to : 80:6F:B0:EE:AA:13 with connection handle 0
Try to connect to : 80:6F:B0:EE:9C:67
Connected to : 80:6F:B0:EE:9C:67 with connection handle None
AOA Params Set for Connection Handle: 0
AOA Callback Set for All Connection Handlers
AOA Started with Connection Handle: 0
Going to sleep for 10 sec
Try to stop AOA result parsing threadSTOP Command Received

AOA Stopped
Master Dis

10.19.190.195 - - [20/Mar/2023 12:40:31] "GET /send/-79 HTTP/1.1" 200 -


Plane Angle: -79
Master : <RTLSNode(CC26x2 Master, started daemon 5232)> 
Passives : [] 
All : [<RTLSNode(CC26x2 Master, started daemon 5232)>]
Devices Reset
Start scan for 5 sec
Scan Results: [{'addr': '80:6F:B0:EE:9C:67', 'addrType': 0, 'rssi': -62, 'advSID': 1, 'periodicAdvInt': 240}, {'addr': '80:6F:B0:EE:AA:13', 'addrType': 0, 'rssi': -65, 'advSID': 1, 'periodicAdvInt': 240}, {'addr': '80:6F:B0:EE:9C:67', 'addrType': 0, 'rssi': -60, 'advSID': 255, 'periodicAdvInt': 0}, {'addr': '80:6F:B0:EE:AA:13', 'addrType': 0, 'rssi': -63, 'advSID': 255, 'periodicAdvInt': 0}]
Try to connect to : 80:6F:B0:EE:9C:67
Connected to : 80:6F:B0:EE:9C:67 with connection handle 0
Try to connect to : 80:6F:B0:EE:AA:13
Connected to : 80:6F:B0:EE:AA:13 with connection handle 1
AOA Params Set for Connection Handle: 0
AOA Params Set for Connection Handle: 1
AOA Callback Set for All Connection Handlers
AOA Started with Connection Handle: 0
AOA Started with Connection Handle: 1
Going to sleep for 10 sec
Try to 

...

Before Turning：{'80:6F:B0:EE:AA:13': [-18, -17, -18, -19, -17, -17, -18, -18, -16, -15, -18, -16, -15, -13, -15, -15, -13, -15, -14, -17, -16, -14], '80:6F:B0:EE:9C:67': [34, 33, 33, 31, 33, 31, 31, 33, 32, 34, 34, 34, 33, 33, 33, 33, 33, 32, 32, 33, 34]}
After Turning： {'80:6F:B0:EE:AA:13': [-22, -20, -22, -22, -21, -20, -21, -22, -19, -19, -21, -20, -20, -22, -22, -20, -19, -20, -19, -19, -20, -21, -19, -19, -20], '80:6F:B0:EE:9C:67': [35, 35, 34, 34, 35, 34, 34, 35, 35, 35, 35, 34, 34, 34, 35, 35, 35, 35, 34, 35, 34, 34, 35, 34]}
Rotation Angle： [-79, -78]
Plane Angle: -77
Master : <RTLSNode(CC26x2 Master, started daemon 2124)> 
Passives : [] 
All : [<RTLSNode(CC26x2 Master, started daemon 2124)>]
Devices Reset
Start scan for 5 sec
Scan Results: [{'addr': '80:6F:B0:EE:AA:13', 'addrType': 0, 'rssi': -78, 'advSID': 1, 'periodicAdvInt': 240}, {'addr': '80:6F:B0:EE:9C:67', 'addrType': 0, 'rssi': -60, 'advSID': 1, 'periodicAdvInt': 240}, {'addr': '80:6F:B0:EE:AA:13', 'addrType': 0, 'rssi

10.19.190.195 - - [20/Mar/2023 12:51:08] "GET /send/-77 HTTP/1.1" 200 -


Plane Angle: -77
Master : <RTLSNode(CC26x2 Master, started daemon 4192)> 
Passives : [] 
All : [<RTLSNode(CC26x2 Master, started daemon 4192)>]
Devices Reset
Start scan for 5 sec
Scan Results: [{'addr': '80:6F:B0:EE:9C:67', 'addrType': 0, 'rssi': -65, 'advSID': 255, 'periodicAdvInt': 0}, {'addr': '80:6F:B0:EE:AA:13', 'addrType': 0, 'rssi': -69, 'advSID': 255, 'periodicAdvInt': 0}, {'addr': '80:6F:B0:EE:9C:67', 'addrType': 0, 'rssi': -58, 'advSID': 1, 'periodicAdvInt': 240}, {'addr': '80:6F:B0:EE:AA:13', 'addrType': 0, 'rssi': -67, 'advSID': 1, 'periodicAdvInt': 240}]
Try to connect to : 80:6F:B0:EE:9C:67
Connected to : 80:6F:B0:EE:9C:67 with connection handle 0
Try to connect to : 80:6F:B0:EE:AA:13
Connected to : 80:6F:B0:EE:AA:13 with connection handle 1
AOA Params Set for Connection Handle: 0
AOA Params Set for Connection Handle: 1
AOA Callback Set for All Connection Handlers
AOA Started with Connection Handle: 0
AOA Started with Connection Handle: 1
Going to sleep for 10 sec
Try to 

10.19.190.195 - - [20/Mar/2023 12:51:49] "GET /send/-77 HTTP/1.1" 200 -
e:\intern_VisBLE\VisBLE_INTERN\AoA Point Cloud\rtls_slave.py:254: RuntimeWarning: invalid value encountered in arccos
  azimuth1 = np.arccos(cosAOAangle1/sinelvation1)
e:\intern_VisBLE\VisBLE_INTERN\AoA Point Cloud\rtls_slave.py:251: RuntimeWarning: invalid value encountered in arccos
  elvation1 = np.arccos((cosAOAangle2-coslevelangle*cosAOAangle1)/sinlevelangle)
10.19.190.195 - - [20/Mar/2023 12:51:55] "GET /calculate/ HTTP/1.1" 200 -


Before Turning：{'80:6F:B0:EE:AA:13': [-22, -20, -22, -22, -21, -20, -21, -22, -19, -19, -21, -20, -20, -22, -22, -20, -19, -20, -19, -19, -20, -21, -19, -19, -20], '80:6F:B0:EE:9C:67': [35, 35, 34, 34, 35, 34, 34, 35, 35, 35, 35, 34, 34, 34, 35, 35, 35, 35, 34, 35, 34, 34, 35, 34]}
After Turning： {'80:6F:B0:EE:AA:13': [-20, -22, -25, -24, -19, -19, -18, -17, -13, -15, -15, -16, -13, -17, -18, -20, -20, -22, -23, -22], '80:6F:B0:EE:9C:67': [35, 33, 32, 34, 34, 33, 34, 35, 35, 34, 33, 34, 34, 33, 33, 34, 34, 34, 34, 34, 34, 33, 35, 35]}
Rotation Angle： [-79, -78, -77]
